<a href="https://colab.research.google.com/github/yellayujin/studying/blob/main/240201_API_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
SERVICE_KEY = os.getenv('SERVICE_KEY')

In [ ]:
URL = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/1/100/'
# json자리에 xml등 넣을 수 있음
# 맨 뒤에 100: 데이터 수 (주의: 한 번에 1000개까지밖에 안 됨)

In [ ]:
import requests
import json
import pandas as pd

req = requests.get(URL)
req.status_code     # 200, req만 입력해도 Response 200나오긴 함

200

In [ ]:
content = req.json()
# content   # 정상 처리되었다고 하고 데이터 쭉
result = pd.DataFrame(content['tbLnOpendataRtmsV']['row'])    #content 안의 tbLn~은 key값. 그 뒤에 row만 찾음 (content 주석 풀면 보일 것)
result

,ACC_YEAR,SGG_CD,SGG_NM,BJDONG_CD,BJDONG_NM,LAND_GBN,LAND_GBN_NM,BONBEON,BUBEON,BLDG_NM,...,OBJ_AMT,BLDG_AREA,TOT_AREA,FLOOR,RIGHT_GBN,CNTL_YMD,BUILD_YEAR,HOUSE_TYPE,REQ_GBN,RDEALER_LAWDNM
0,2024,11380,은평구,10700,응암동,1,대지,0091,0007,CJ드림시티,...,12900,24.84,34.07,15.0,,,2004,오피스텔,중개거래,서울 은평구
1,2024,11140,중구,14500,광희동1가,1,대지,0089,0001,광희동금호트윈 오피스텔,...,9850,21.96,31.13,14.0,,,1999,오피스텔,중개거래,서울 중구
2,2024,11740,강동구,10900,천호동,1,대지,0039,0009,미래위너스,...,26400,29.96,19.16,4.0,,,2016,연립다세대,중개거래,서울 강동구
3,2024,11710,송파구,11300,거여동,1,대지,0605,0000,휴먼빌 시티,...,18000,24.15,35.41,4.0,,,2020,오피스텔,중개거래,서울 송파구
4,2024,11530,구로구,10200,구로동,1,대지,0044,0008,비즈트위트 오렌지,...,15000,20.80,31.43,8.0,,,2013,오피스텔,중개거래,서울 영등포구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2024,11380,은평구,10900,신사동,1,대지,0005,0042,삼원제우스(5-42),...,30000,45.64,26.54,4.0,,,2016,연립다세대,직거래,
96,2024,11140,중구,16200,신당동,1,대지,0846,0000,신당동파라다이스,...,82000,84.77,0.00,6.0,,,2000,아파트,중개거래,서울 중구
97,2024,11500,강서구,10300,화곡동,1,대지,0056,0323,화성빌라A동(56-323),...,8000,28.53,13.35,4.0,,,2001,연립다세대,직거래,
98,2024,11215,광진구,10500,자양동,1,대지,0610,0027,(610-27),...,23800,39.27,25.62,1.0,,,1986,연립다세대,중개거래,서울 광진구


In [ ]:
# json 대신 xml로 해보기!
URL

'http://openapi.seoul.go.kr:8088/5a736a6857736b7236387253467746/json/tbLnOpendataRtmsV/1/100/'

In [ ]:
URL1 = URL.replace('json', 'xml')
URL1

'http://openapi.seoul.go.kr:8088/5a736a6857736b7236387253467746/xml/tbLnOpendataRtmsV/1/100/'

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

req = requests.get(URL1)
# req.content       # 약간 html과 비슷한 형태로 나옴
soup = BeautifulSoup(req.content, 'lxml')   # soup객체로 바꾸어줘야 진짜 html느낌 됨
# print(soup.prettify()) # prettify()가 뭐야?

C:\Users\skrtk\AppData\Local\Temp\ipykernel_20140\859826875.py:7: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(req.content, 'lxml')   # soup객체로 바꾸어줘야 진짜 html느낌 됨


In [ ]:
# try 이 부분 로직 정리해보자!

col = ['acc_year', 'sgg_cd', 'sgg_nm', 'bjdong_cd', 'land_gbn', 'land_gbn_nm', 'bonbeon', 'bubeon',
       'bldg_nm', 'deal_ymd', 'house_type']

result_dict = {colname: [] for colname in col}

for colname in col:
    for tag in soup.find_all(colname):
        result_dict[colname].append(tag.text)
result = pd.DataFrame(result_dict)
result

,acc_year,sgg_cd,sgg_nm,bjdong_cd,land_gbn,land_gbn_nm,bonbeon,bubeon,bldg_nm,deal_ymd,house_type
0,2024,11380,은평구,10700,1,대지,0091,0007,CJ드림시티,20240131,오피스텔
1,2024,11140,중구,14500,1,대지,0089,0001,광희동금호트윈 오피스텔,20240131,오피스텔
2,2024,11740,강동구,10900,1,대지,0039,0009,미래위너스,20240131,연립다세대
3,2024,11710,송파구,11300,1,대지,0605,0000,휴먼빌 시티,20240131,오피스텔
4,2024,11530,구로구,10200,1,대지,0044,0008,비즈트위트 오렌지,20240131,오피스텔
...,...,...,...,...,...,...,...,...,...,...,...
95,2024,11380,은평구,10900,1,대지,0005,0042,삼원제우스(5-42),20240129,연립다세대
96,2024,11140,중구,16200,1,대지,0846,0000,신당동파라다이스,20240129,아파트
97,2024,11500,강서구,10300,1,대지,0056,0323,화성빌라A동(56-323),20240129,연립다세대
98,2024,11215,광진구,10500,1,대지,0610,0027,(610-27),20240129,연립다세대


In [ ]:
# 공공데이터 포털- 국토교통부
PUBLIC_SERVICE_KEY = os.getenv('PUBLIC_SERVICE_KEY')
URL2 = 'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?_wadl&type=xml'
        # end point에서 가져왔는데, 공식문서에서 서비스 URL가져오는 것이 더 바람직

pageNo = 1 # 페이지 번호
numOfRows = 500 # 한 페이지 결과수
LAWD_CD = 11500 # 지역코드
DEAL_YMD = 202101 # 계약월
url = f'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey={PUBLIC_SERVICE_KEY}&pageNo={pageNo}&numOfRows={numOfRows}&LAWD_CD={LAWD_CD}&DEAL_YMD={DEAL_YMD}'
req = requests.get(url)
req

<Response [200]>

In [ ]:
req.text    # 인증키 아직 인증 못 받으면 뜨는 에러: <?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>99</resultCode><resultMsg>SERVICE ACCESS DENIED ERROR.</resultMsg></header></response>'

In [ ]:
# xml 데이터를 dictionary(json) 타입으로 변경할 수 있는 라이브러리

In [ ]:
pip install xmltodict

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 위에서 인증 못 받아서 일단 필기
contents = xmltodict.parse(req.text)
data = pd.DataFrame(contents['response']['body']['items']['item'])
data.head()


NameError: name 'xmltodict' is not defined

In [ ]:
# 1000개 이상으로 다중 데이터 수집
URL1000 = URL.replace('100', '1000')
URL1000

In [ ]:
# 합쳐보기
df = None
for i in range(1000, 4000, 1000):
    URL1000 = f'http://openapi.seoul.go.kr:8088/{SERVICE_KEY}/json/tbLnOpendataRtmsV/{(i-1000)+1}/{i}/'
    print(URL1000)

    req = requests.get(URL1000)
    content = req.json()

    result = pd.DataFrame(content['tbLnOpendataRtmsV']['row'])
    df = pd.concat([df, result], ignore_index=True)

In [ ]:
df